# CRPS & Bayesian Regression

In this notebook, we will show a simple example of the usage of the CRPS, using a Bayesian Ridge Regression.

The predictions are the μ and σ parameter's of the posterior-predictive distribution.

We will show the usage of both the analytical CRPS for the normal distribution, and the non-parameteric CRPS, with simulations.

In [1]:
import numpy as np
from scipy import stats

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge


Here we implement both the NRG and PWM forms of the CRPS (Zamo & Naveau, 2017) and the analytical solution for the normal distribution (Taillardat, Zamo & Naveau, 2016)

In [2]:
# Adapted to numpy from pyro.ops.stats.crps_empirical
# Copyright (c) 2017-2019 Uber Technologies, Inc.
# SPDX-License-Identifier: Apache-2.0
def crps_nrg(y_true, y_pred, sample_weight=None):
    num_samples = y_pred.shape[0]
    absolute_error = np.mean(np.abs(y_pred - y_true), axis=0)

    y_pred = np.sort(y_pred, axis=0)
    diff = y_pred[1:] - y_pred[:-1]
    weight = np.arange(1, num_samples) * np.arange(num_samples - 1, 0, -1)
    weight = np.expand_dims(weight, -1)

    per_obs_crps = absolute_error - np.sum(diff * weight, axis=0) / num_samples**2
    return np.average(per_obs_crps, weights=sample_weight)


def crps_pwm(y_true, y_pred, sample_weight=None):
    num_samples = y_pred.shape[0]
    absolute_error = np.mean(np.abs(y_pred - y_true), axis=0)

    y_pred = np.sort(y_pred, axis=0)
    b0 = y_pred.mean(axis=0)
    b1_values = y_pred * np.arange(num_samples).reshape((num_samples, 1))
    b1 = b1_values.mean(axis=0) / num_samples

    per_obs_crps = absolute_error + b0 - 2 * b1
    return np.average(per_obs_crps, weights=sample_weight)


def crps_gaussian(x, mu, sig, sample_weight=None):
    sx = (x - mu) / sig
    pdf = stats.norm.pdf(sx)
    cdf = stats.norm.cdf(sx)
    per_obs_crps = sig * (sx * (2 * cdf - 1) + 2 * pdf - 1. / np.sqrt(np.pi))
    return np.average(per_obs_crps, weights=sample_weight)


load the data, split to train-test, train and predict the test set and generate simulations.

In [3]:
dataset = fetch_california_housing()
x_train, x_test, y_train, y_test = train_test_split(
    dataset.data, dataset.target, test_size=0.2, random_state=17
)

model = BayesianRidge()
model.fit(x_train, y_train)
pred_mean, pred_std = model.predict(x_test, return_std=True)

# create simulation-predictions
predictions = stats.multivariate_normal.rvs(pred_mean, np.diag(pred_std ** 2), 10_000, 17)


In [5]:
print(f'The CRPS with the PWM implementation is: {crps_pwm(y_test, predictions)}')

print(f'The CRPS with the NRG implementation is: {crps_nrg(y_test, predictions)}')

# in this simple case, we can use the analytic solution of CRPS for normal distribution:
print(f'The CRPS with the analytical solution is: {crps_gaussian(y_test, pred_mean, pred_std)}')


The CRPS with the PWM implementation is: 0.41025561010581335
The CRPS with the NRG implementation is: 0.4100499323798249
The CRPS with the analytical solution is: 0.41000374217408847
